In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

2024-03-30 13:03:59.955512: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-30 13:04:02.742010: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-30 13:04:02.742122: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-30 13:04:02.742286: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-30 13:04:03.160979: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-30 13:04:10.066552: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT

In [41]:
X = pd.read_csv('MovieData/small_movies_X.csv')
Y = pd.read_csv('MovieData/small_movies_Y.csv')
R = pd.read_csv('MovieData/small_movies_R.csv')
W = pd.read_csv('MovieData/small_movies_W.csv')
B = pd.read_csv('MovieData/small_movies_b.csv')
movie_list = pd.read_csv('MovieData/small_movie_list.csv')

In [42]:
Y.max()

0.0        5.0
0.0.1      5.0
0.0.2      0.5
0.0.3      5.0
0.0.4      5.0
          ... 
0.0.433    5.0
0.0.434    5.0
0.0.435    5.0
0.0.436    5.0
0.0.437    5.0
Length: 443, dtype: float64

In [43]:
"""
import os
directory = '/path/to/csv/files'

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        filepath = os.path.join(directory, filename)
        >>trim file name from .csv then use it
        f'{filename}' = pd.read_csv(filepath)
"""

'\nimport os\ndirectory = \'/path/to/csv/files\'\n\nfor filename in os.listdir(directory):\n    if filename.endswith(".csv"):\n        filepath = os.path.join(directory, filename)\n        >>trim file name from .csv then use it\n        f\'{filename}\' = pd.read_csv(filepath)\n'

In [44]:
print('Y:',Y.shape, 'R:',R.shape)
print('X:',X.shape)
print('n_user:',Y.shape[1])
print('n_movie:',Y.shape[0])
print('n:',X.shape[1])
print('W:',W.shape)
print('B',B.shape)

Y: (4777, 443) R: (4777, 443)
X: (4777, 10)
n_user: 443
n_movie: 4777
n: 10
W: (442, 10)
B (1, 443)


In [45]:
W.loc[442] = W.iloc[0]
W.shape

(443, 10)

In [46]:
B.columns = W.index
B

,0,1,2,3,4,5,6,7,8,9,...,433,434,435,436,437,438,439,440,441,442
0,0.236095,0.356534,0.1282,0.315502,0.236364,-0.229178,-0.144015,0.075727,-0.370031,-0.021354,...,-0.176575,-0.473329,-0.361718,0.084787,0.429559,-0.405086,0.45839,0.412608,-0.36224,-0.32014


In [47]:
X_r = np.array(X)
R_r = np.array(R)
Y_r = np.array(Y)
W_r = np.array(W)
B_r = np.array(B)

In [48]:
'''
#LOOP IMPLEMENTATION
def cost_fun(X, W, B, Y, R, lambda_):
    nm, nu = Y.shape
    J_cost = 0
    for j in range(nu):
        w = W[j,:]
        b = B[0,j]
        for i in range(nm):
            J_cost += (R[i,j]*(np.dot(w, X[i]) + b - Y[i,j]))**2
    J_cost += lambda_*(np.sum(W*W) + np.sum(X*X))
    J_cost /= 2
    return J_cost
'''
#cost_fun(X_r, W_r, B_r, Y_r, R_r, .2)

'\n#LOOP IMPLEMENTATION\ndef cost_fun(X, W, B, Y, R, lambda_):\n    nm, nu = Y.shape\n    J_cost = 0\n    for j in range(nu):\n        w = W[j,:]\n        b = B[0,j]\n        for i in range(nm):\n            J_cost += (R[i,j]*(np.dot(w, X[i]) + b - Y[i,j]))**2\n    J_cost += lambda_*(np.sum(W*W) + np.sum(X*X))\n    J_cost /= 2\n    return J_cost\n'

In [49]:
#VECTOR IMPLEMENTATION USING NUMPY
def cost_funV(X, W, B, Y, R, lambda_):
    """
    Y: (4777, 443)
    R: (4777, 443)
    X: (4777, 10)
    n_user: 443
    n_movie: 4777
    n: 10
    W: (443, 10)
    B (1, 443)
    """
    J = 0.5*np.sum(R*(np.dot(X, W.T) + B - Y)**2)
    J += 0.5*lambda_*(np.sum(W*W )+ np.sum(X*X))
    return J

In [50]:
cost_funV(X_r,W_r,B_r,Y_r,R_r,.2)

276621.6848353459

In [51]:
#VECTOR IMPLEMENTATION USING TENSORFLOW
def cofi_cost_func_v(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    Vectorized for speed. Uses tensorflow operations to be compatible with custom training loop.
    Args:
      X (ndarray (num_movies,num_features)): matrix of item features
      W (ndarray (num_users,num_features)) : matrix of user parameters
      b (ndarray (1, num_users)            : vector of user parameters
      Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float): regularization parameter
    Returns:
      J (float) : Cost
    """
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

In [52]:
cofi_cost_func_v(X_r, W_r, B_r, Y_r, R_r, .2)

<tf.Tensor: shape=(), dtype=float64, numpy=276621.6848353459>

In [138]:
#Rating By a new user:
my_ratings = np.zeros(Y_r.shape[0])

my_ratings[2700] = 5 
my_ratings[2609] = 2
my_ratings[929]  = 5   
my_ratings[246]  = 5   
my_ratings[2716] = 3  
my_ratings[1150] = 5  
my_ratings[382]  = 2 
my_ratings[366]  = 5  
my_ratings[622]  = 5   
my_ratings[988]  = 3   
my_ratings[2925] = 1  
my_ratings[2937] = 1  
my_ratings[793]  = 5
my_rated = [i for i in range(len(my_ratings)) if my_ratings[i] > 0]
#13 movies were rated

In [ ]:
# Reload ratings and add new ratings
Y_r = np.c_[my_ratings, Y_r]
R_r = np.c_[(my_ratings != 0).astype(int), R_r]

In [55]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
tr = R_r*Y_r
scalar.fit(tr)
Y_norm = scalar.fit_transform(tr)

In [89]:
# Normalize the Dataset
print('Y_r:',Y_r.shape)
print('R_r:',R_r.shape)

Y_r: (4777, 444)
R_r: (4777, 444)


In [90]:
# Set Initial Parameters (W, X), use tf.Variable to track these variables

n_movie, n_user = Y_r.shape
n_feature = X_r.shape[1]

W = tf.Variable(tf.random.normal((n_user,  n_feature),dtype=tf.float64),  name='W')
B = tf.Variable(tf.random.normal((1, n_user),dtype=tf.float64),  name='B')

X = tf.Variable(X_r,  name='X')

optimizer = keras.optimizers.Adam(learning_rate= 0.01)

In [140]:
optimizer = keras.optimizers.Adam(learning_rate= 1e-1)

In [141]:
iterations = 201
lambda_ = 1
for iter in range(iterations):
    # Use TensorFlow’s GradientTape to record the operations used to compute the cost 
    with tf.GradientTape() as tape:
        # Compute the cost (forward pass included in cost)
        cost_value = cofi_cost_func_v(X, W, B, Y_r, R_r, lambda_)

    # Use the gradient tape to automatically retrieve the gradients of the trainable variables with respect to the loss
    grads = tape.gradient( cost_value, [X, W, B] )

    # Run one step of gradient descent by updating the value of the variables to minimize the loss.
    optimizer.apply_gradients( zip(grads, [X, W, B]) )

    # Log periodically.
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

Training loss at iteration 0: 7300.7
Training loss at iteration 20: 6460.9
Training loss at iteration 40: 5698.4
Training loss at iteration 60: 5283.6
Training loss at iteration 80: 5165.6
Training loss at iteration 100: 5128.9
Training loss at iteration 120: 5103.1
Training loss at iteration 140: 5084.2
Training loss at iteration 160: 5072.5
Training loss at iteration 180: 5065.2
Training loss at iteration 200: 5059.9


<h3>Making Prediction</h3>

In [100]:
mean = scalar.mean_.reshape(1,-1)

In [142]:
predicted_rating = np.dot(np.array(X), np.array(W).T) + np.array(B)
(predicted_rating).max()

9.26250631755876

In [143]:
predicted_rating

array([[3.04373947, 4.44477699, 4.29219759, ..., 2.15548354, 3.80269838,
        2.99656796],
       [4.00370076, 4.47895016, 3.76645832, ..., 2.80078199, 3.5105092 ,
        2.29248964],
       [3.01538141, 4.49104567, 3.71298849, ..., 2.29938477, 3.28348144,
        2.0291407 ],
       ...,
       [3.72652247, 4.67140295, 3.96375834, ..., 2.93067721, 3.91447009,
        3.58996425],
       [3.72652247, 4.67140296, 3.96375834, ..., 2.93067722, 3.91447009,
        3.58996426],
       [3.72652247, 4.67140295, 3.96375834, ..., 2.93067721, 3.91447009,
        3.58996425]])

In [144]:
movie_list

,Unnamed: 0,mean rating,number of ratings,title
0,0,3.400000,5,"Yards, The (2000)"
1,1,3.250000,6,Next Friday (2000)
2,2,2.000000,4,Supernova (2000)
3,3,2.000000,4,Down to You (2000)
4,4,2.672414,29,Scream 3 (2000)
...,...,...,...,...
4773,4773,3.500000,1,Jon Stewart Has Left the Building (2015)
4774,4774,4.000000,1,Black Butler: Book of the Atlantic (2017)
4775,4775,3.500000,1,No Game No Life: Zero (2017)
4776,4776,3.500000,1,Flint (2017)


In [150]:
filter=(movie_list["number of ratings"] > 10)
my_predicted_rating = predicted_rating[:,0]
indx = np.argsort(-my_predicter_rating)


movie_list.loc[indx].loc[filter].head(20)

,Unnamed: 0,mean rating,number of ratings,title
2914,2914,3.514706,34,Thor (2011)
392,392,3.205882,17,Kate & Leopold (2001)
928,928,3.375000,12,Girl with a Pearl Earring (2003)
579,579,3.900000,25,Secretary (2002)
4552,4552,3.633333,15,Get Out (2017)
729,729,3.140625,32,Phone Booth (2002)
987,987,3.906250,16,Dawn of the Dead (2004)
154,154,2.607143,14,Vertical Limit (2000)
4,4,2.672414,29,Scream 3 (2000)
1481,1481,3.633333,30,Wallace & Gromit in The Curse of the Were-Rabb...


In [151]:
movie_list[filter]

,Unnamed: 0,mean rating,number of ratings,title
4,4,2.672414,29,Scream 3 (2000)
5,5,4.220930,43,"Boondock Saints, The (2000)"
7,7,3.062500,32,"Beach, The (2000)"
10,10,3.714286,21,Boiler Room (2000)
12,12,3.564103,39,Pitch Black (2000)
...,...,...,...,...
4631,4631,3.423077,13,Dunkirk (2017)
4653,4653,3.805556,18,Blade Runner 2049 (2017)
4668,4668,3.538462,13,Coco (2017)
4691,4691,3.125000,12,Star Wars: The Last Jedi (2017)


In [153]:
import pickle

variables = {'W': W, 'X': X, 'B': B}
with open('MovieRecommender', 'wb') as f:
    pickle.dump(variables, f)